Setup

In [33]:
#%% Packages 
import wrds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 6]

import time
from time import strptime, strftime

# Setups
pd.set_option("display.max_rows", 200) # max number or rows to be displayed 
#%% Set WRDS Connection
db = wrds.Connection(wrds_username='zrsong') # make sure to configure wrds connector before hand.


Loading library list...
Done


Load in Compustat Data

In [20]:
compm_tables = db.list_tables(library="comp") # compd: Compustat daily update 

['aco_amda', 'aco_imda', 'aco_indfnta', 'aco_indfntq', 'aco_indfntytd', 'aco_indsta', 'aco_indstq', 'aco_indstytd', 'aco_notesa', 'aco_notesq', 'aco_notessa', 'aco_notesytd', 'aco_pnfnda', 'aco_pnfndq', 'aco_pnfndytd', 'aco_pnfnta', 'aco_pnfntq', 'aco_pnfntytd', 'aco_transa', 'aco_transq', 'aco_transsa', 'aco_transytd', 'adsprate', 'asec_amda', 'asec_imda', 'asec_notesa', 'asec_notesq', 'asec_transa', 'asec_transq', 'bank_aacctchg', 'bank_adesind', 'bank_afnd1', 'bank_afnd2', 'bank_afnddc1', 'bank_afnddc2', 'bank_afntind', 'bank_funda', 'bank_funda_fncd', 'bank_fundq', 'bank_fundq_fncd', 'bank_iacctchg', 'bank_idesind', 'bank_ifndq', 'bank_ifndytd', 'bank_ifntq', 'bank_ifntytd', 'bank_names', 'bank_namesq', 'chars', 'co_aacctchg', 'co_aaudit', 'co_adesind', 'co_adjfact', 'co_afnd1', 'co_afnd2', 'co_afnddc1', 'co_afnddc2', 'co_afntind1', 'co_afntind2', 'co_ainvval', 'co_amkt', 'co_busdescl', 'co_cotype', 'co_filedate', 'co_fortune', 'co_hgic', 'co_iacctchg', 'co_iaudit', 'co_idesind', '

In [14]:
# import thomson reuters 13f data from WRDS
# tr_13f: Thomson Reuters 13F Holdings Data
# tr_13f_tables = db.list_tables(library="tr_13f")

In [39]:
fund_table = 'fundq'

varlist = ['conm', 'tic', 'cusip','fyearq', 'fqtr', 'fyr', 'atq','capxy', 'ceqq', 'cogsq', 
           'cshoq', 'dlcq', 'dlcchy','dlttq', 'dpq', 'ibq', 'itccy', 'fic',
           'ltq', 'mibq', 'niq', 'prstkccy', 'pstkq', 'req', 'revtq', 'saleq',
           'seqq', 'txdbq', 'txdiq', 'txditcq', 'wcapchy', 'xinty', 'xrdq', 'xsgaq', 'capr1q',
           'mkvaltq', 'epspxq', 'epsfxq', 'ajexq', 'prccq', 'oancfy', 'ivncfy', 'rdq', 'prstkcy', 'sstky', 'tstkq', 'dvpy', 'dvy', 'cdvcy']


query = """SELECT gvkey, datadate, {}
           FROM comp.{}
           WHERE datafmt = 'STD'
           AND popsrc = 'D'
           AND indfmt = 'INDL'
           AND consol = 'C'
           AND fyearq <= 2023
           AND fyearq >= 1988;""".format(", ".join(varlist), fund_table)

compq = db.raw_sql(query, date_cols=['datadate'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wrds/sql.py:580: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, chunk])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wrds/sql.py:580: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, chunk])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wrds/sql.py:580: FutureWarning: The behavior of DataFrame 

In [44]:
# check gvkey -- 3243 (C - Citibank)
citibank = compq[compq['tic'] == 'C']
citibank.describe()
compq.describe()

,datadate,fyearq,fqtr,fyr,atq,capxy,ceqq,cogsq,cshoq,dlcq,...,ajexq,prccq,oancfy,ivncfy,prstkcy,sstky,tstkq,dvpy,dvy,cdvcy
count,1619291,1.619291e+06,1.618944e+06,1.619291e+06,1.255353e+06,1.158256e+06,1.246334e+06,1.297816e+06,1.444915e+06,1.180134e+06,...,1.619286e+06,1.265252e+06,1.142650e+06,1.142717e+06,1.108990e+06,1.151901e+06,1.240860e+06,1.159367e+06,1.160576e+06,24525.000000
mean,2006-08-15 01:32:41.474003200,2.006037e+03,2.482153e+00,1.019899e+01,7.820551e+03,1.031515e+02,1.254041e+03,3.935636e+02,1.858238e+02,6.626297e+02,...,3.155311e+01,2.631450e+01,1.887571e+02,-1.675043e+02,4.195921e+01,2.617266e+01,1.857796e+02,2.257442e+00,4.497888e+01,161.568241
min,1987-09-30 00:00:00,1.988000e+03,1.000000e+00,1.000000e+00,-1.069690e+02,-3.258000e+03,-1.399650e+05,-3.641500e+04,0.000000e+00,-1.068000e+03,...,1.000000e-08,1.000000e-06,-1.405370e+05,-3.132910e+05,-1.820800e+01,-1.630334e+03,-1.573000e+03,-1.018000e+03,-1.631969e+03,0.000000
25%,1997-12-31 00:00:00,1.997000e+03,1.000000e+00,9.000000e+00,2.488000e+01,1.090000e-01,7.503000e+00,1.560000e+00,5.847000e+00,4.000000e-03,...,1.000000e+00,2.250000e+00,-1.174000e+00,-3.778200e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.697000
50%,2006-06-30 00:00:00,2.006000e+03,2.000000e+00,1.200000e+01,1.902240e+02,1.714000e+00,6.132000e+01,1.281600e+01,1.855200e+01,1.849000e+00,...,1.000000e+00,9.812500e+00,1.449000e+00,-2.797000e+00,0.000000e+00,8.100000e-02,0.000000e+00,0.000000e+00,0.000000e+00,41.000000
75%,2015-06-30 00:00:00,2.015000e+03,3.000000e+00,1.200000e+01,1.290108e+03,1.682800e+01,3.836220e+02,9.880125e+01,5.688100e+01,2.311300e+01,...,1.000000e+00,2.337500e+01,3.416275e+01,-2.700000e-02,8.800000e-02,2.780000e+00,2.580000e-01,0.000000e+00,1.900000e+00,146.000000
max,2024-05-31 00:00:00,2.023000e+03,4.000000e+00,1.200000e+01,4.329380e+06,6.364500e+04,5.612730e+05,1.890810e+05,2.597497e+07,6.142374e+05,...,5.454545e+05,1.416000e+05,2.317860e+05,5.400500e+05,9.562500e+04,7.749000e+04,2.549170e+05,8.541900e+04,8.245200e+04,22135.000000
std,NaN,1.020479e+01,1.116537e+00,3.175853e+00,7.812745e+04,7.425116e+02,7.581328e+03,2.511666e+03,4.333110e+04,9.763478e+03,...,4.044116e+03,5.862995e+02,1.910857e+03,2.830683e+03,5.442468e+02,3.515272e+02,2.691644e+03,1.702763e+02,3.635009e+02,423.656489


In [35]:
# check whether gvkey datadate is a unique key
compq.duplicated(['gvkey', 'datadate']).sum()
compq[compq.duplicated(['gvkey', 'fyearq', 'fqtr'])]
# How to deal with duplicates? Keep last available entry (datadate)
compq.dropna(subset=['fyearq','fqtr'], inplace=True)
compq.sort_values(['gvkey','fyearq','fqtr','atq'], inplace=True)
compq = compq[~compq.duplicated(['gvkey', 'fyearq','fqtr'], keep='last')]
compq.duplicated(['gvkey', 'fyearq','fqtr']).sum()

# output pickle format
compq.to_pickle("compustat_quarterly.pkl")

In [36]:
# keep only naics code 52 (merge with compa annual data to get naics code) using compa_annual.pkl in the same folder
compa = pd.read_pickle("../Data/compa_annual.pkl")
compa = compa[['gvkey', 'fyear', 'naicsh', 'prstkc', 'sstk','dvc','tstk', 'capr1']]
# merge compq and compa
# rename fyear fyearq to merge
compa.rename(columns={'fyear':'fyearq'}, inplace=True)
compq1 = pd.merge(compq, compa, how='left', on=['gvkey', 'fyearq'])
# keep only those with six-digit naics code that starts with 52 (change to string first)
compq1['naicsh'] = compq1['naicsh'].astype(str)
compq1 = compq1[compq1['naicsh'].str.startswith('52')]


In [37]:
# number of observations in compq
compq1.shape[0]
compq1.duplicated(['gvkey', 'fyearq','fqtr']).sum()

0

In [38]:
# save compq to csv file
compq1.to_csv("../Data/compustat_quarterly.csv", index=False)